In [1]:
import os
import time
import re
import json
from PIL import Image
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [2]:
# path = "/root/chrome_driver/chromedriver"
path_window = r"/Users/seungsu/Desktop/YBIGTA/chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=2400,1080")
# chrome_options.add_argument("headless")

driver = webdriver.Chrome(path_window, options=chrome_options)
driver.implicitly_wait(3)

#### 아이템 태그 리스트 얻기

In [5]:
p=2
test_url = category_url_list["knit"]["url"][:63] + f"page={p}&" + category_url_list["knit"]["url"][63:]
driver.get(test_url)
items = []
time.sleep(2)
ul = driver.find_element_by_css_selector("ul._53c8ee")
all_items = ul.find_elements_by_css_selector("li._c29d78._d85b45")
for item in all_items:
    box = item.find_element_by_css_selector("div._8eb9f8._a57b85")
    
    item_url = item.find_element_by_css_selector("a").get_attribute("href")
#     img = item.find_element_by_css_selector("img._fc8ffc").get_attribute("src")
    item_brand = box.find_element_by_css_selector("h3._346238").text
    item_name = box.find_element_by_css_selector("p._d85b45").text
    
    items.append({"brand":item_brand, "name":item_name, "href":item_url})
items

[{'brand': 'Gucci',
  'name': 'GG V-넥 스웨터',
  'href': 'https://www.farfetch.com/kr/shopping/men/gucci-gg-v-item-14020527.aspx?storeid=9359'},
 {'brand': 'Gucci',
  'name': '구찌 테니스 후디',
  'href': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14024216.aspx?storeid=9359'},
 {'brand': 'Mastermind Japan',
  'name': '스컬 로고 프린트 후디',
  'href': 'https://www.farfetch.com/kr/shopping/men/mastermind-japan--item-14948597.aspx?storeid=9359'},
 {'brand': 'Z Zegna',
  'name': '래글런 후디',
  'href': 'https://www.farfetch.com/kr/shopping/men/z-zegna--item-14459350.aspx?storeid=9359'},
 {'brand': 'Reigning Champ',
  'name': '풀오버 테리 후디',
  'href': 'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107342.aspx?storeid=9359'},
 {'brand': 'BOSS',
  'name': '지데오 롤 넥 스웨터',
  'href': 'https://www.farfetch.com/kr/shopping/men/boss--item-14545288.aspx?storeid=9359'},
 {'brand': 'Reigning Champ',
  'name': '크루 넥 스웨트셔츠',
  'href': 'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-141

In [6]:
for i in items:
    print(i)

{'brand': 'Gucci', 'name': 'GG V-넥 스웨터', 'href': 'https://www.farfetch.com/kr/shopping/men/gucci-gg-v-item-14020527.aspx?storeid=9359'}
{'brand': 'Gucci', 'name': '구찌 테니스 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14024216.aspx?storeid=9359'}
{'brand': 'Mastermind Japan', 'name': '스컬 로고 프린트 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/mastermind-japan--item-14948597.aspx?storeid=9359'}
{'brand': 'Z Zegna', 'name': '래글런 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/z-zegna--item-14459350.aspx?storeid=9359'}
{'brand': 'Reigning Champ', 'name': '풀오버 테리 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107342.aspx?storeid=9359'}
{'brand': 'BOSS', 'name': '지데오 롤 넥 스웨터', 'href': 'https://www.farfetch.com/kr/shopping/men/boss--item-14545288.aspx?storeid=9359'}
{'brand': 'Reigning Champ', 'name': '크루 넥 스웨트셔츠', 'href': 'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107351.aspx?storeid=9359'}
{'brand': 'Monc

#### 세부 데이터 URL

In [7]:
item_detail_url = items[4]["href"]
item_detail_url

'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107342.aspx?storeid=9359'

#### 세부 데이터 크롤링

In [8]:
# 데이터 저장 dict
detail_item_info = {}

# 접속
driver.get(item_detail_url)
time.sleep(2)
soup = bs(driver.page_source, 'html.parser')
print(soup)
# 기본정보
detail_item = driver.find_element_by_css_selector("div._9d3f24._da3196")
basic_info = driver.find_element_by_css_selector("div._c40757")
img_list = driver.find_element_by_css_selector("div._cc7da7").find_elements_by_css_selector("img._f3a89c")
category_box = driver.find_element_by_css_selector("ol._4e9dce")

detail_item_info["shop"] = "farfetch"
detail_item_info["brand"] = basic_info.find_element_by_css_selector("span._e87472._346238._e4b5ec").text
detail_item_info["product"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").text
detail_item_info["product_id"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").get_attribute("data-attribute")
detail_item_info["url"] = item_detail_url
detail_item_info["sub_category"] = category_box.find_elements_by_css_selector("li._9d7c74")[-1].find_element_by_css_selector("span").text
# detail_item_info["category"] = 
# detail_item_info["id"] = 
print(img_list)
# 이미지url
detail_item_img = []
for img in img_list:
    detail_item_img.append(img.get_attribute("src"))
detail_item_info["img"] = detail_item_img

detail_item_info
# 상세정보
# infos = driver.find_element_by_css_selector("div._9d3f24._da3196").find_elements_by_css_selector("div._4919a3._da3196")
# for row in infos:
#     pass

<html class=" js flexbox no-touch cssanimations csstransitions video svg placeholder haspointerevents csstransitionend not-ie win chrome chrome83 no-mobile-device win-chrome83" dir="LTR" lang="ko" style="--vh:949px;"><head> <title>Reigning Champ 풀오버 테리 후디 할인 L 세일로 만나보는 파페치 해외 브랜드 컬렉션 ✈ 관부가세 포함 금액, 무료 반품</title> <meta content="Reigning Champ 풀오버 테리 후디 세일! 빠른 배송과 무료 반품 혜택까지 파페치에서 실속있게 쇼핑하세요. 해외 디자이너 브랜드 컬렉션을 한국어로 쉽게 쇼핑해보세요." name="description"/> <meta content="7jRAROVKvQYssFeSePEX2e9IQoHjq84OOvlu6i4hKN0" name="google-site-verification"/> <link href="https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107342.aspx" rel="canonical"/> <link href="https://www.farfetch.com/shopping/men/reigning-champ--item-14107342.aspx" hreflang="en-US" rel="alternate"/><link href="https://www.farfetch.com/ae/shopping/men/reigning-champ--item-14107342.aspx" hreflang="en-AE" rel="alternate"/><link href="https://www.farfetch.com/ao/shopping/men/reigning-champ--item-14107342.aspx" hreflang="en-AO" re

[<selenium.webdriver.remote.webelement.WebElement (session="402fc690e4f3a65c12276e36d11623cb", element="28c51cd2-2c8f-434a-9eeb-044f20554b8c")>, <selenium.webdriver.remote.webelement.WebElement (session="402fc690e4f3a65c12276e36d11623cb", element="0b91a5e0-0aa5-414a-ad92-c84be550e8d7")>, <selenium.webdriver.remote.webelement.WebElement (session="402fc690e4f3a65c12276e36d11623cb", element="979f3ef7-1576-4a2e-8dfb-c029ebfa156f")>, <selenium.webdriver.remote.webelement.WebElement (session="402fc690e4f3a65c12276e36d11623cb", element="90df2959-fc47-4191-813e-f5fa71889ce5")>, <selenium.webdriver.remote.webelement.WebElement (session="402fc690e4f3a65c12276e36d11623cb", element="4ff998b6-75b1-47ea-8f86-6c7358e7521f")>]


{'shop': 'farfetch',
 'brand': 'Reigning Champ',
 'product': '풀오버 테리 후디',
 'product_id': '14107342',
 'url': 'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107342.aspx?storeid=9359',
 'sub_category': '후디',
 'img': ['https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268861_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268868_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268874_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268881_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268886_1000.jpg']}


# 개별 함수

### farfetch URL 만들기!!

In [4]:
category_url_list = {"knit" : {"category" : "knit", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136334"},
                    "denim" : {"category" : "denim", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136337"},
                    "shirts" : {"category" : "shirts", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136331"}, 
                    "shorts" : {"category" : "shorts", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136339"}, 
                    "suit" : {"category" : "suit", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136340"},
                    "jacket" : {"category" : "jacket", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136335"},
                    "coat" : {"category" : "coat", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136336"},
                    "t-shirt" : {"category" : "t-shirt", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136332"},
                    "pants" : {"category" : "pants", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136338"},
                    "polo-shirt" : {"category" : "polo-shirt", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136333"}
                    }

### 아이템리스트(by 페이지, 카테고리) 반환 함수

In [9]:
def getItemList(sub_category, page= 1):
#     self.category_url_list
    itemList_url = category_url_list[sub_category]["url"][:63] + f"page={str(page)}&" + category_url_list[sub_category]["url"][63:]
    driver.get(itemList_url)
    items = []
    time.sleep(2)
    ul = driver.find_element_by_css_selector("ul._53c8ee")
    all_items = ul.find_elements_by_css_selector("li._c29d78._d85b45")
    for item in all_items:
        box = item.find_element_by_css_selector("div._8eb9f8._a57b85")

        item_url = item.find_element_by_css_selector("a").get_attribute("href")
    #     img = item.find_element_by_css_selector("img._fc8ffc").get_attribute("src")
        item_brand = box.find_element_by_css_selector("h3._346238").text
        item_name = box.find_element_by_css_selector("p._d85b45").text

        items.append({"brand":item_brand, "name":item_name, "href":item_url})
    items
    return items

In [10]:
a = ['knit',
 'denim',
 'shirts',
 'shorts',
 'suit',
 'jacket',
 'coat',
 't-shirt',
 'pants',
 'polo-shirt']
# getItemList("knit", 1)

### 단일페이지 크롤러

In [11]:
def crawl_item(item_detail_url):
    # 데이터 저장 dict
    detail_item_info = {}

    # 접속
    driver.get(item_detail_url)
    time.sleep(2)
    soup = bs(driver.page_source, 'html.parser')
    
    # 기본정보
    detail_item = driver.find_element_by_css_selector("div._9d3f24._da3196")
    basic_info = driver.find_element_by_css_selector("div._c40757")
    img_list = driver.find_element_by_css_selector("div._cc7da7").find_elements_by_css_selector("img._f3a89c")
    category_box = driver.find_element_by_css_selector("ol._4e9dce")

    detail_item_info["shop"] = "farfetch"
    detail_item_info["brand"] = basic_info.find_element_by_css_selector("span._e87472._346238._e4b5ec").text
    detail_item_info["product"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").text
    detail_item_info["product_id"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").get_attribute("data-attribute")
    detail_item_info["url"] = item_detail_url
    detail_item_info["sub_category"] = category_box.find_elements_by_css_selector("li._9d7c74")[-1].find_element_by_css_selector("span").text
    # detail_item_info["category"] = 
    # detail_item_info["id"] = 
    
    # 이미지url
    detail_item_img = []
    for img in img_list:
        detail_item_img.append(img.get_attribute("src"))
    detail_item_info["img"] = detail_item_img

    detail_item_info
    # 상세정보
    # infos = driver.find_element_by_css_selector("div._9d3f24._da3196").find_elements_by_css_selector("div._4919a3._da3196")
    # for row in infos:
    #     pass    
    return detail_item_info

In [12]:
crawl_item(item_detail_url)

{'shop': 'farfetch',
 'brand': 'Reigning Champ',
 'product': '풀오버 테리 후디',
 'product_id': '14107342',
 'url': 'https://www.farfetch.com/kr/shopping/men/reigning-champ--item-14107342.aspx?storeid=9359',
 'sub_category': '후디',
 'img': ['https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268861_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268868_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268874_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268881_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/10/73/42/14107342_20268886_1000.jpg']}

# 병합

### 고유 id 생성코드

In [15]:
def make_id(num,shop="13", category="00"):
#     10 : 무신사
#     11 : 29cm
#     12 : oco
#     13 : farfetch
#     14 : matchesfashion
    item_id = shop + category + str(num).rjust(5, "0")
    return item_id

### 한페이지 모든 데이터 크롤링 함수

In [48]:
# 아이템 dict을 저장할 dict
item_group = dict()
num = 1
for item in items:
    item_id = make_id(num, "13", "01")
    item_group[item_id] = crawl_item(item["href"])
    num += 1
item_group

KeyboardInterrupt: 

In [53]:
item_group

{'130100001': {'shop': 'farfetch',
  'brand': 'KENZO',
  'product': '파리 로고 인타르시아 니트 후디',
  'product_id': '13901899',
  'url': 'https://www.farfetch.com/kr/shopping/men/kenzo--item-13901899.aspx?storeid=9359',
  'sub_category': '후디',
  'img': ['https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001820_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001825_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001830_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001835_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001838_1000.jpg']},
 '130100002': {'shop': 'farfetch',
  'brand': 'STONE ISLAND',
  'product': '고스트 하프 후드 집업',
  'product_id': '15332629',
  'url': 'https://www.farfetch.com/kr/shopping/men/stone-island--item-15332629.aspx?storeid=9359',
  'sub_category': '후디',
  'img': ['https://cdn-images.farfetch-contents.com/15/33/26/29/15332629_27139344_1000

### 카테고리 크롤링 함수

In [16]:
def crawl_category(sub_category, page, path=None ,save=False):
    # 아이템 dict을 저장할 dict
    item_group = dict()
    num = 1
    
    # page 별
    for p in range(1, int(page + 1)):
        # 페이지 item리스트 뽑기
        itemList = getItemList(sub_category, p)
        
        # 개별 아이템 json화
        for item in itemList:
            item_id = make_id(num, "13", "01")
            a = crawl_item(item["href"])
            item_group[item_id] = a
            print(a)
            num += 1
    
    # json으로 저장
    if save == True:
        # path 설정
        if path == None:
            path = f"./itemList.json"
        else:
            path = path + "/itemList.json"
        with open(path, 'w', encoding='utf-8') as make_file:
            json.dump(item_group, make_file, ensure_ascii=False, indent="\t")
    
    return item_group

In [17]:
crawl_category("knit", page=1, save=True)

{'shop': 'farfetch', 'brand': 'Billionaire Boys Club', 'product': '로고 프린트 스웨트셔츠', 'product_id': '15359302', 'url': 'https://www.farfetch.com/kr/shopping/men/billionaire-boys-club--item-15359302.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/15/35/93/02/15359302_27768033_1000.jpg', 'https://cdn-images.farfetch-contents.com/15/35/93/02/15359302_27768037_1000.jpg']}


WebDriverException: Message: chrome not reachable
  (Session info: chrome=83.0.4103.116)


### 저장용 디렉토리 생성

In [5]:
def check_directory(category_name, folder_path=None):
    if folder_path == None:
        folder_path = "C:/Users/seungsu/Desktop/ICT-project/crawling"
    try:
        os.mkdir(f"{folder_path}/{category_name}/")
        print("Directory is created!")
    except:
        print("Directory exist!")